In [3]:
import numpy as np
import pandas as pd
import gc; gc.enable()
import random, time, datetime

np.random.seed(1122)

# Read extra datasets
st = time.time()

dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8', 'onpromotion':str}

items = pd.read_csv('../input/items.csv', dtype={'perishable':str})
stores = pd.read_csv('../input/stores.csv')
transactions = pd.read_csv('../input/transactions.csv', parse_dates=['date'])
holidays = pd.read_csv('../input/holidays_events.csv', dtype={'transferred':bool}, parse_dates=['date'])
oil = pd.read_csv('../input/oil.csv', parse_dates=['date'])
#train = pd.read_csv('../input/train.csv', dtype=dtypes, parse_dates=['date'])

print("{0:.3f} MB".format(items.memory_usage(index=True).sum()/1E6))
print("{0:.3f} MB".format(stores.memory_usage(index=True).sum()/1E6))
print("{0:.3f} MB".format(transactions.memory_usage(index=True).sum()/1E6))
print("{0:.3f} MB".format(holidays.memory_usage(index=True).sum()/1E6))
print("{0:.3f} MB".format(oil.memory_usage(index=True).sum()/1E6))
#print("{0:.3f} MB".format(train.memory_usage(index=True).sum()/1E6))

def print_dataframe_size(df):
    print("{0:.3f} MB".format(df.memory_usage(index=True).sum()/1E6))
    
def print_runtime(start, end):
    print("runtime: {}".format( datetime.timedelta(seconds=(end-start)/60)))

    
print_runtime(st, time.time())

print_dataframe_size(items)
print_dataframe_size(stores)
print_dataframe_size(transactions)
print_dataframe_size(oil)
#print_dataframe_size(train)

0.131 MB
0.002 MB
2.004 MB
0.014 MB
0.020 MB
runtime: 0:00:00.000937
0.131 MB
0.002 MB
2.004 MB
0.020 MB


In [2]:
#train = train[(train['date'].dt.year == 2017)]
#print_dataframe_size(train)

In [4]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [4]:
stores = pd.get_dummies(stores, columns = ['type', ] )
stores.head()

,store_nbr,city,state,cluster,type_A,type_B,type_C,type_D,type_E
0,1,Quito,Pichincha,13,0,0,0,1,0
1,2,Quito,Pichincha,13,0,0,0,1,0
2,3,Quito,Pichincha,8,0,0,0,1,0
3,4,Quito,Pichincha,9,0,0,0,1,0
4,5,Santo Domingo,Santo Domingo de los Tsachilas,4,0,0,0,1,0


In [5]:
holidays = pd.read_csv('../input/holidays_events.csv', dtype={'transferred':bool}, parse_dates=['date'])
#data['hol']['transferred'].map({'False': False, 'True': True})
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [6]:
stores['city'].unique()

array(['Quito', 'Santo Domingo', 'Cayambe', 'Latacunga', 'Riobamba',
       'Ibarra', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil', 'Salinas',
       'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad', 'Cuenca',
       'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'], dtype=object)

In [7]:
holidays['locale_name'].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato', 'Ibarra',
       'Quevedo', 'Santo Domingo de los Tsachilas', 'Santa Elena', 'Quito',
       'Loja', 'Salinas', 'Guayaquil'], dtype=object)

In [8]:
holidays.loc[lambda df: (df.description.str.contains('Independencia de Guayaquil')),]

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
190,2015-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,False
274,2016-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,False
331,2017-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,False


In [9]:
# First attempt to use the holidays
# Create a binary field for national holidays which is expected to affect overall sales
# In a next step, regional holidays should be introduced
holidays['national_holiday'] = False 
holidays.loc[lambda df: (df.type=='Holiday') & (df.locale=='National') & ~(df.transferred), 'national_holiday']=True
holidays.loc[lambda df: (df.type=='Transfer') & (df.locale=='National'), 'national_holiday']=True

holidays.loc[lambda df: (df.description.str.contains('Independencia de Guayaquil')),]

,date,type,locale,locale_name,description,transferred,national_holiday
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,False
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False,True
190,2015-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,False,True
274,2016-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,False,True
331,2017-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,False,True


In [10]:
holidays[['date','national_holiday']].head()

,date,national_holiday
0,2012-03-02,False
1,2012-04-01,False
2,2012-04-12,False
3,2012-04-14,False
4,2012-04-21,False


In [12]:
items['perishable_w'] = items['perishable'].map({False:1.0, True:1.25}) 
items.head()

,item_nbr,family,class,perishable,perishable_w
0,96995,GROCERY I,1093,False,1.00
1,99197,GROCERY I,1067,False,1.00
2,103501,CLEANING,3008,False,1.00
3,103520,GROCERY I,1028,False,1.00
4,103665,BREAD/BAKERY,2712,True,1.25
